In [0]:
!pip install librosa --upgrade

In [0]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import IPython.display as ipd
import copy
import time

!gdown https://drive.google.com/uc?id=110yz4UGIbveLW8ENFI1DjypNXs96q0FZ

y, sr = librosa.load('/content/ex.wav',sr=16000) #Читаем файл
ipd.Audio(y, rate=sr) 

# Теория

In [0]:
class Hparams():
  def __init__(self, sr):
    self.n_fft = int(sr * 25 / 1000)
    self.hop_length=int(sr * 10 / 1000)
    self.n_mels = 40
    self.n_iter = 10
    self.win_length = 20

## Получить  спектр

In [0]:
hp = Hparams(sr)
# Получаем спектр
spectr = librosa.stft(y=y,hop_length=hp.hop_length,n_fft=hp.n_fft)
# Амплитуда
ampl = np.abs(spectr) 
# Фаза
phasa = np.angle(spectr)
# Исходный сигнал только по амплитуде
y_ampl = librosa.istft(ampl,hop_length=hp.hop_length)
ipd.Audio(y_ampl, rate=sr) 

## Вернуть фазу

In [0]:
# Добавляем фазу
ampl_phasa =  ampl*np.cos(phasa) + 1j*ampl*np.sin(phasa)
# Исходный сигнал по амплитуде и фазе
y_ampl_phasa = librosa.istft(ampl_phasa,hop_length=hp.hop_length)
ipd.Audio(y_ampl_phasa, rate=sr) 

## Получить мелспектрограмму

In [0]:
melspectr = librosa.feature.melspectrogram( #Мелспектр
    y,
    sr,
    n_fft=hp.n_fft,
    hop_length=hp.hop_length,
    n_mels=hp.n_mels
)

#Для отображения
S_dB = librosa.power_to_db(melspectr, ref=np.max)
plt.figure(figsize=(10, 4))
cmap = cm.get_cmap('viridis')
librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', cmap=cmap)
plt.colorbar()
plt.title('Melspectrogram')
plt.tight_layout()
plt.show()

## Мелспектрограмма в аудио

In [0]:
ampl_mel = librosa.feature.inverse.mel_to_stft(melspectr, sr, n_fft=hp.n_fft)
y_mel_ampl = librosa.istft(ampl_mel,hop_length=hp.hop_length)
ipd.Audio(y_mel_ampl, rate=sr) 

## Гриффин-Лим

In [0]:
y_griffinlim = librosa.griffinlim(ampl_mel, hop_length=hp.hop_length)
ipd.Audio(y_griffinlim, rate=sr) 

## Вернуть к мел фазу

In [0]:
ampl_phasa =  ampl_mel*np.cos(phasa) + 1j*ampl_mel*np.sin(phasa)
y_mel_ampl_phasa = librosa.istft(ampl_phasa,hop_length=hp.hop_length)
ipd.Audio(y_mel_ampl_phasa, rate=sr) 

## Почему мы в синтезаторе отбрасываем фазу

### Амплитуда несёт информацию

In [0]:
Xdb = 10.0 * np.log10(ampl**2 + 1e-10)
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

### Фаза случайна

In [0]:
librosa.display.specshow(phasa, sr=sr, x_axis='time', y_axis='hz')

В фазе нет патерна, который нейронная сеть может хорошо достать

## Отобразить все графики

In [0]:
plt.figure(figsize=(16, 12))
ax = plt.subplot(5,1,1)
librosa.display.waveplot(y, sr=sr, color='b')
plt.title('Оригинал')
plt.xlabel('')
plt.subplot(5,1,2, sharex=ax, sharey=ax)
librosa.display.waveplot(y_ampl, sr=sr, color='r')
plt.title('Только Амплитуда')
plt.xlabel('')
plt.subplot(5,1,3, sharex=ax, sharey=ax)
librosa.display.waveplot(y_mel_ampl, sr=sr, color='r')
plt.title('Только Амплитуда, полученная из Mel')
plt.xlabel('')
plt.subplot(5,1,4, sharex=ax, sharey=ax)
librosa.display.waveplot(y_griffinlim, sr=sr, color='g')
plt.title('Только Амплитуда, полученная из Mel + griffinlim')
plt.xlabel('')
plt.subplot(5,1,5, sharex=ax, sharey=ax)
librosa.display.waveplot(y_mel_ampl_phasa, sr=sr, color='r')
plt.title('Амплитуда и фаза, полученная из Mel')
plt.tight_layout()
plt.show()

# MelGan

In [0]:
!git clone https://github.com/vlomme/MelGan-WavGan.git

In [0]:
cd MelGan-WavGan/

In [0]:
!gdown https://drive.google.com/uc?id=154hohXOGS2UauTt-o7kNvkcXFKpGXufa
!unzip logs.zip

In [0]:
!python melgan.py -r g -f wav

### Оригинальный сигнал

In [0]:
y_orig, sr = librosa.load('./gen/orig/RUSLAN.wav',sr=16000)
ipd.Audio(y_orig, rate=sr) 

### Гриффин Лим

In [0]:
y_griffinlim, sr = librosa.load('./gen/RUSLAN.wav',sr=16000)
ipd.Audio(y_griffinlim, rate=sr) 

### Вокодер MelGan

In [0]:
y_melgan, sr = librosa.load('./gen/output/mel_RUSLAN.wav',sr=16000)
ipd.Audio(y_melgan, rate=sr) 

# WavGan

Идея. Зачем синтезировать из мелспектрограммы, если можно применить Гриффин лим и получить сигнал и останется его только подправить

In [0]:
!python wavgan.py -r g -f wav

In [0]:
y_wavgan, sr = librosa.load('./gen/output/wav_RUSLAN.wav',sr=16000)
ipd.Audio(y_wavgan, rate=sr) 

# TTS

In [0]:
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0
!sudo apt-get install portaudio19-dev
!pip install PyAudio
!pip install sounddevice
!pip install unidecode

!gdown https://drive.google.com/uc?id=19Gl9ejMUm_ll6cKozt5N99qDBN25FXDA
!unzip test.zip

In [0]:
cd test/

In [0]:
!python synthesize.py --text_list 'Привет сотрудники вконтакте. Как у вас дела.' --checkpoint male
#!python synthesize.py --text_list 'Привет сотрудники вконтакте. Как у вас дела.' --checkpoint female
#!python synthesize.py --text_list 'Привет сотрудники вконтакте. Как у вас дела.' --checkpoint zueva

In [0]:
y_tac, sr = librosa.load('./tacotron_output/wav-batch_0_sentence_0-mel.wav',sr=16000)
ipd.Audio(y_tac, rate=sr) 

In [0]:
!python wavgan.py -r g -f wav

In [0]:
y_tac, sr = librosa.load('./tacotron_output/output/wav_wav-batch_0_sentence_0-mel.wav',sr=16000)
ipd.Audio(y_tac, rate=sr) 